In [1]:
import time
import numpy as np

import pybullet as p
from surrol.utils.pybullet_utils import (
    reset_camera,
)
from surrol.robots.psm import Psm

p.connect(p.GUI)
# p.connect(p.DIRECT)
p.setGravity(0, 0, -9.81)

pybullet build time: Feb 15 2025 12:06:37


Version = 4.1 Metal - 89.4
Vendor = Apple
Renderer = Apple M1
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [2]:
# PsmEnv     : [[ 2.5    3.   ] [-0.25   0.25 ] [ 3.426  3.776]]
# NeedleReach: [[ 2.5    3.   ] [-0.25   0.25 ] [ 3.426  3.776]]

# Actually workspace lowest Z should be 3.476, due to the tray pad.

# Workspace limits: [[ 2.5    3.   ] [-0.25   0.25 ] [ 3.426  3.776]]
# Scaling factor: 5.0
# Pose of tray: ((0.55, 0, 0.6751), (0, 0, 0))

POSE_TRAY = ((0.55, 0, 0.6751), (0, 0, 0))
SCALING = 5.
workspace_limits = np.array([[2.5, 3.], [-0.25, 0.25], [3.426, 3.776]])  # x, y, z

In [3]:
obj_id_5 = p.loadURDF('/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/tray/tray_pad.urdf',
                    np.array(POSE_TRAY[0]) * 5.0,
                    p.getQuaternionFromEuler(POSE_TRAY[1]),
                    globalScaling=5.0)

In [ ]:
p.removeBody(obj_id_5)

In [5]:
reset_camera(yaw=90, pitch=-20, dist=1.0, target=np.array(POSE_TRAY[0]) * SCALING)

In [6]:
POSE_PSM1 = ((0.05, 0.24, 0.8524), (0, 0, -(90 + 20) / 180 * np.pi))
# POSE_PSM1 = ((0, 0, 0), (0, 0, 0))

psm = Psm(POSE_PSM1[0], 
          p.getQuaternionFromEuler(POSE_PSM1[1]), 
          scaling=5.0)

In [ ]:
p.removeBody(psm.body)

In [ ]:
psm.pose_rcm2world(psm.get_current_position(), 'tuple')


((3.0, 0.2500000000000001, 3.7760000228881836),
 (0.6324555320336759,
  0.6324555320336759,
  -0.3162277660168379,
  -0.31622776601683794))

TypeError: tuple indices must be integers or slices, not str

In [8]:
print(workspace_limits)

pos = (
    workspace_limits[0][1],
    workspace_limits[1][1],
    workspace_limits[2][1]
)

orn = (1.0, 1.0, -0.5, -0.5)
# orn = (0.5, 0.5, -0.5, -0.5)
joint_positions = psm.inverse_kinematics((pos, orn), psm.EEF_LINK_INDEX)
print(np.rad2deg(joint_positions[5]))
psm.reset_joint(joint_positions)

[[ 2.5    3.   ]
 [-0.25   0.25 ]
 [ 3.426  3.776]]
-10.040864127967499


array([-0.10206997, -0.78161303,  0.14310819, -0.34265871,  0.12602561,
       -0.17524614])

In [53]:
angle = joint_positions[5]
T_joint = np.array([
    [np.cos(angle), -np.sin(angle), 0, 0],
    [np.sin(angle), np.cos(angle), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])
print(T_joint)
np.linalg.inv(T_joint)

[[ 0.98468365  0.17435053  0.          0.        ]
 [-0.17435053  0.98468365  0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


array([[ 0.98468365, -0.17435053,  0.        ,  0.        ],
       [ 0.17435053,  0.98468365,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [54]:
T_joint_inv = np.array([
    [np.cos(-angle), -np.sin(-angle), 0, 0],
    [np.sin(-angle), np.cos(-angle), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

T_joint_inv

array([[ 0.98468365, -0.17435053,  0.        ,  0.        ],
       [ 0.17435053,  0.98468365,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [22]:
from surrol.utils.pybullet_utils import get_joint
get_joint(psm.body, "psm_tool_yaw_joint")

5

In [55]:
from surrol.utils.pybullet_utils import (
    get_link_pose
)

tool_yaw_pos = np.array(get_link_pose(psm.body, 5)[0])
tool_pitch_pos = np.array(get_link_pose(psm.body, 4)[0])

print("yaw:", tool_yaw_pos)
print("pitch:", tool_pitch_pos)
print("diff:", tool_yaw_pos - tool_pitch_pos)

yaw: [3.         0.25       3.77600002]
pitch: [2.97311807 0.25793296 3.81184244]
diff: [ 0.02688193 -0.00793296 -0.03584242]


In [ ]:
from surrol.utils.pybullet_utils import (
    get_euler_from_quaternion
)

tool_yaw_orn = get_euler_from_quaternion(get_link_pose(psm.body, 5)[1])
tool_pitch_orn = get_euler_from_quaternion(get_link_pose(psm.body, 4)[1])

print("yaw:", tool_yaw_orn)
print("pitch:", tool_pitch_orn)

In [56]:
from surrol.utils.robotics import get_matrix_from_pose_2d, get_matrix_from_euler, get_pose_2d_from_matrix
T_child_world = get_matrix_from_pose_2d(get_link_pose(psm.body, 5))
T_child_world

array([[ 0.        ,  0.6       , -0.8       ,  3.        ],
       [ 1.        ,  0.        ,  0.        ,  0.25      ],
       [ 0.        , -0.8       , -0.6       ,  3.77600002],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [58]:
T_parent_world = get_matrix_from_pose_2d(get_link_pose(psm.body, 4))
T_parent_world

array([[ 5.90810187e-01, -8.00000005e-01,  1.04610301e-01,
         2.97311807e+00],
       [-1.74350626e-01, -9.38816552e-08,  9.84683634e-01,
         2.57932961e-01],
       [-7.87746902e-01, -5.99999993e-01, -1.39480557e-01,
         3.81184244e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [59]:
T_child_parent = np.array([
    [0, 1, 0, 0.0455],
    [0, 0, 1, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1]
])

T_pw = T_child_world @ np.linalg.inv(T_joint) @ np.linalg.inv(T_child_parent)
T_pw

array([[ 0.59081019, -0.8       ,  0.10461032,  2.97311814],
       [-0.17435053,  0.        ,  0.98468365,  0.25793295],
       [-0.78774692, -0.6       , -0.13948042,  3.81184251],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
get_pose_2d_from_matrix(T_parent_world @ np.linalg.inv(T_pw))

((1.4873344728824078, 1.2185358340593364, -0.537597419008244),
 (0.18485783937431616,
  -0.18485783519931728,
  0.09242892420191066,
  0.9607872053468234))

In [9]:
scale = SCALING * 5.0

hemisphere_pos = [
    workspace_limits[0].mean(), 
    workspace_limits[1].mean(), 
    workspace_limits[2][0] - 0.016 # 0.0102 * 2.5
]

print(hemisphere_pos)

hemisphere_id = p.loadURDF(
    '/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/sphere/half_sphere.urdf',
    np.array(hemisphere_pos),
    p.getQuaternionFromEuler((-np.pi / 2, 0, 0)),
    useFixedBase=False,
    globalScaling=scale
)

[2.75, 0.0, 3.41]


In [ ]:
# Get the shape data of hemisphere
shape_data = p.getVisualShapeData(hemisphere_id)
print("Shape data:", shape_data)

# scale = 5, number = 0.05
# scale = 25, number = 0.25

In [ ]:
p.removeBody(hemisphere_id)

In [ ]:
yaw = (np.random.rand() - 0.5) * np.pi
needle_pos = (
    workspace_limits[0].mean() + (np.random.rand() - 0.5) * 0.1,
    workspace_limits[1].mean() + (np.random.rand() - 0.5) * 0.1,
    workspace_limits[2][0]
)
print("Needle pos:", needle_pos)
print("Needle yaw:", yaw / np.pi * 180)
needle_id = p.loadURDF(
    '/Users/kantaphat/Research/DEX/SurRoL/surrol/assets/needle/needle_40mm.urdf',
    np.array(needle_pos),
    p.getQuaternionFromEuler((0, 0, yaw)),
    useFixedBase=False,
    globalScaling=SCALING
)

In [ ]:
p.removeBody(needle_id)

In [ ]:
# Get position of the needle_id
needle_pos, needle_orn = p.getBasePositionAndOrientation(needle_id)
print("Needle pos:", needle_pos)
print("Needle orn:", needle_orn)

In [ ]:
len(p.getVisualShapeData(hemisphere_id)[0])

In [10]:
# Vector from hemisphere center to PSM position
print("Pos:", pos)
print("Hemisphere pos:", hemisphere_pos)
pc = np.array(pos) - np.array(hemisphere_pos)
print("PC:", pc)
unit_pc = pc / np.linalg.norm(pc)
print("Unit PC:", unit_pc)
hemisphere_radius = p.getVisualShapeData(hemisphere_id)[0][3][0]
print("Hemisphere radius:", hemisphere_radius)
cylinder_pos = hemisphere_pos + unit_pc * hemisphere_radius
print("Cylinder pos:", cylinder_pos)

default_axis = np.array([0, 0, 1])

axis_of_rotation = np.cross(default_axis, unit_pc)
angle_of_rotation = np.arccos(np.dot(default_axis, unit_pc))

# Create quaternion from Euler angles
cylinder_orn = p.getQuaternionFromAxisAngle(axis_of_rotation, angle_of_rotation)
print("Cylinder orn:", cylinder_orn)


Pos: (3.0, 0.25, 3.776)
Hemisphere pos: [2.75, 0.0, 3.41]
PC: [0.25  0.25  0.366]
Unit PC: [0.49127766 0.49127766 0.7192305 ]
Hemisphere radius: 0.25
Cylinder pos: [2.87281942 0.12281942 3.58980763]
Cylinder orn: (-0.26493844389915466, 0.26493844389915466, 0.0, 0.9271543622016907)


In [11]:
cylinder_visual_id = p.createVisualShape(
    p.GEOM_CYLINDER, 
    radius=0.05, 
    length=0.05,
    rgbaColor=[0, 1, 1, 0.5]
)

cylinder_id = p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=cylinder_visual_id,
    basePosition=cylinder_pos,
    baseOrientation=cylinder_orn
)

In [ ]:
p.removeBody(cylinder_id)

In [ ]:
cylinder_center_shape = p.createVisualShape(
            p.GEOM_SPHERE, 
            radius=0.007, 
            rgbaColor=[1.0, 0.41, 0.71, 1]
        )
        
cylinder_center_id = p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=cylinder_center_shape,
    basePosition=cylinder_pos
)

In [ ]:
cameraTargetPosition=np.array((0.55, -0.52, 0.375)) * SCALING
# reset_camera(yaw=-90, pitch=-30, dist=0.7, target=np.array(POSE_TRAY[0]) * SCALING)
reset_camera(yaw=90, pitch=-89, dist=0.7, target=np.array(POSE_TRAY[0]) * SCALING)

In [ ]:
orn = (0.5, 0.5, -0.5, -0.5)
joint_positions = psm.inverse_kinematics((cylinder_pos, orn), psm.EEF_LINK_INDEX)
psm.reset_joint(joint_positions)

In [ ]:
from surrol.utils.pybullet_utils import (
    get_link_pose
)

pose_world = get_link_pose(psm.body, psm.DoF - 1)[0]
pose_world

In [ ]:
circle_shape = p.createVisualShape(
    p.GEOM_SPHERE, 
    radius=0.01, 
    rgbaColor=[1.0, 0.41, 0.71, 1]
)
        
circle_id = p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=circle_shape,
    basePosition=get_link_pose(psm.body, 4)[0]
)


In [ ]:
p.removeBody(circle_id)

In [ ]:
reset_camera(yaw=90, pitch=-30, dist=0.4, target=get_link_pose(psm.body, 4)[0])

In [12]:
while True:
    p.stepSimulation()
    time.sleep(1.0/240.0)

KeyboardInterrupt: 

In [ ]:
def draw_workspace(limits, color=(1, 0, 0), line_width=2):
    """Draw a wireframe box representing the workspace limits."""
    x_min, x_max = limits[0]
    y_min, y_max = limits[1]
    z_min, z_max = limits[2]
    
    # Define the 8 corners of the box
    corners = [
        [x_min, y_min, z_min],
        [x_min, y_min, z_max],
        [x_min, y_max, z_min],
        [x_min, y_max, z_max],
        [x_max, y_min, z_min],
        [x_max, y_min, z_max],
        [x_max, y_max, z_min],
        [x_max, y_max, z_max]
    ]
    
    # Define the 12 edges of the box
    edges = [
        (0, 1), (0, 2), (0, 4),
        (1, 3), (1, 5),
        (2, 3), (2, 6),
        (3, 7),
        (4, 5), (4, 6),
        (5, 7),
        (6, 7)
    ]
    
    # Draw each edge
    for edge in edges:
        p.addUserDebugLine(
            corners[edge[0]],
            corners[edge[1]],
            lineColorRGB=color,
            lineWidth=line_width,
            lifeTime=0
        )
        
draw_workspace(workspace_limits)

In [ ]:
psm.pose_rcm2world(psm.get_current_position(), 'tuple')[0]

In [ ]:
reset_camera(yaw=90, pitch=-10, dist=1.2, 
             target=np.array(psm.pose_rcm2world(psm.get_current_position(), 'tuple')[0]))

In [ ]:
sphere_shape = p.createVisualShape(
    p.GEOM_SPHERE, 
    radius=0.005, 
    rgbaColor=[0, 1, 1, 1]
)

sphere_body = p.createMultiBody(
    baseMass=0,
    baseVisualShapeIndex=sphere_shape,
    basePosition=psm.pose_rcm2world(psm.get_current_position(), 'tuple')[0]
)

In [ ]:
p.removeBody(sphere_body)

In [ ]:
p.disconnect()